In [115]:
import numpy as np
import pandas as pd
import emoji
import re
import pickle

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split  # Import for splitting the data
from gensim.models import KeyedVectors
from datetime import datetime

np.random.seed(0)


In [2]:
data = pd.read_csv("data/emoji_data/emoji_data.csv", header=None)
data.head()

,0,1
0,Woke up feelin' dangerous,5
1,Got my mind on my money,5
2,Haha that was hilarious,1
3,Feeling like Uptown Funk,5
4,Congratulations for having a baby,2


Emoji Dictionary

In [3]:
"""emoji_dictionary = {
    0: ":red_heart:",  # Love
    1: ":face_with_tears_of_joy:",  # Laughter
    2: ":smiling_face_with_heart-eyes:",  # Adoration
    3: ":loudly_crying_face:",  # Sadness
    4: ":fire:",  # Excitement
    5: ":thumbs_up:",  # Approval
    6: ":folded_hands:",  # Gratitude
    7: ":angry_face:",  # Anger
    8: ":sparkles:",  # Happiness
    9: ":weary_face:",  # Exhaustion
    10: ":astonished_face:",  # Surprise
    11: ":confused_face:",  # Confusion
    12: ":tropical_drink:",  # Celebration
    13: ":broken_heart:",  # Heartbreak
    14: ":thinking_face:",  # Contemplation
    15: ":sleeping_face:",  # Sleepiness
    16: ":victory_hand:",  # Success
    17: ":thumbs_down:",  # Disapproval
    18: ":grimacing_face:",  # Discomfort
    19: ":smiling_face_with_halo:",  # Innocence
}"""

emoji_dictionary = {
    0: ":red_heart:",  # Love #
    1: ":face_with_tears_of_joy:",  # Laughter
    2: ":grinning_face_with_big_eyes:", # Happiness #
    3: ":loudly_crying_face:",  # Sadness #
    4: ":smiling_face_with_heart-eyes:",  # Adoration
    5: ":fire:",  # Excitement
    6: ":thumbs_up:",  # Approval
    7: ":folded_hands:",  # Gratitude
    8: ":angry_face:",  # Anger
    9: ":thinking_face:",  # Contemplation
}


Function to get labels from CLDR names

In [4]:
def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

Reading data

In [5]:
data = pd.read_csv('data/emoji_data/emoji_data.csv', header=None)
data.head()

,0,1
0,Woke up feelin' dangerous,5
1,Got my mind on my money,5
2,Haha that was hilarious,1
3,Feeling like Uptown Funk,5
4,Congratulations for having a baby,2


In [6]:

def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

In [7]:
X = data[0].values
Y = data[1].values


Embeddings

With glove dataset

In [8]:
# with open('data/glove_dataset/glove.6B.100d.txt','r', encoding='utf8') as file:
#     content = file.readlines()

In [9]:
# embeddings = {}

# for line in content:
#     line = line.split()
#     embeddings[line[0]] = np.array(line[1:], dtype=float)

With crawl dataset

In [10]:
fasttext_model_path = "data/fast_text/crawl_dataset/crawl-300d-2M-subword.vec"
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path)


In [11]:
# Convert FastText embeddings to a dictionary
embeddings = {}
for word in fasttext_model.index_to_key:
    embeddings[word] = fasttext_model.get_vector(word)
    
    

Convert input text into tokens

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word_to_index = tokenizer.word_index

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
Xtokens = tokenizer.texts_to_sequences(X)


In [14]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    
    return maxlen
maxlen = get_maxlen(Xtokens)



In [15]:
Xtrain = pad_sequences(Xtokens, maxlen=maxlen, padding='post', truncating='post')


In [16]:
Ytrain = to_categorical(Y)

Split data into train and test set

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(Xtrain, Ytrain, test_size=0.2, random_state=42)


Model

In [18]:
embed_size = 300

In [19]:
embedding_matrix = np.zeros((len(word_to_index) + 1, embed_size))

for word, i in word_to_index.items():
    if word in embeddings:
        embed_vector = embeddings[word]
        embedding_matrix[i] = embed_vector
    else:
        # Handle out-of-vocabulary words or phrases by aggregating subword embeddings
        phrase_embed_sum = None
        clean_word = re.split(r'\W+', word)
        for subword in clean_word:
            if subword in embeddings:
                if phrase_embed_sum is None:
                    phrase_embed_sum = embeddings[subword].copy()
                else:
                    phrase_embed_sum += embeddings[subword]
        # Assign the aggregated embedding to a new array with the desired dtype
        if phrase_embed_sum is not None:
            embedding_matrix[i] = phrase_embed_sum / len(clean_word)
            

            

In general more layers >>

In [116]:

epochs = 1000


#units
layers = [256, 16, 4, 2]

num_layers = len(layers)


num_classes = len(emoji_dictionary)

Code to generate file name for saving model

In [117]:
def returnfilename():
    layers_str = '_'.join(map(str, layers))  # Convert layers to string and join them with '_'
    return f"epochs_{epochs}_layers_{layers_str}"


In [118]:
model = Sequential()

# Add Embedding layer
model.add(Embedding(input_dim=len(word_to_index) + 1,
                    output_dim=embed_size,
                    input_length=maxlen,
                    weights=[embedding_matrix],
                    trainable=False))


# Add LSTM layers
for i in range(num_layers):
    # For the last layer, return_sequences=False
    if i == num_layers - 1:
        model.add(LSTM(units=layers[i]))
    else:
        model.add(LSTM(units=layers[i], return_sequences=True))

# Add output Dense layer
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [119]:
model.fit(Xtrain, Ytrain, epochs=epochs,  verbose = 1)

Epoch 1/1000
24/24 [==============================] - 5s 17ms/step - loss: 2.2933 - accuracy: 0.1335
Epoch 2/1000
24/24 [==============================] - 0s 18ms/step - loss: 2.2412 - accuracy: 0.1990
Epoch 3/1000
24/24 [==============================] - 0s 20ms/step - loss: 2.2118 - accuracy: 0.1819
Epoch 4/1000
24/24 [==============================] - 0s 19ms/step - loss: 2.1824 - accuracy: 0.1937
Epoch 5/1000
24/24 [==============================] - 0s 19ms/step - loss: 2.1693 - accuracy: 0.2304
Epoch 6/1000
24/24 [==============================] - 0s 18ms/step - loss: 2.1351 - accuracy: 0.2343
Epoch 7/1000
24/24 [==============================] - 0s 19ms/step - loss: 2.1071 - accuracy: 0.2330
Epoch 8/1000
24/24 [==============================] - 0s 17ms/step - loss: 2.0793 - accuracy: 0.2552
Epoch 9/1000
24/24 [==============================] - 0s 16ms/step - loss: 2.0702 - accuracy: 0.2605
Epoch 10/1000
24/24 [==============================] - 0s 17ms/step - loss: 2.0624 - accura

In [120]:
filename = returnfilename()

In [121]:
model_save_path = "models/training set v.3.0/"

In [122]:

model_json = model.to_json()
with open(f"{model_save_path}/json/{filename}.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(f"{model_save_path}/weights/{filename}.h5")



In [123]:
metrics_folder = f"{model_save_path}/metrics"

In [124]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime('%d %B, %Y')

# Write model architecture and details to file
with open(f"{metrics_folder}/{filename}.md", "w", encoding="utf-8") as file:
    file.write("# Model description:<br>\n")
    file.write(f"Date generated: {formatted_datetime}<br>\n")
    file.write(f"Number of classes: {num_classes}<br>\n")
    file.write(f"Number of samples: {len(X)}<br>\n")
    file.write(f"Training set size: {len(X_train)}<br>\n")
    file.write(f"Test set size: {len(X_test)}<br>\n")
    file.write(f"Epochs used: {epochs}<br>\n")
    file.write(f"Number of layers: {num_layers}<br>\n")
    file.write(f"Layers used: {layers}<br>\n")
    file.write("Optimizer: Adam<br>\n")
    file.write("Loss function: Categorical Crossentropy<br>\n")
    file.write("# Evaluation Results<br>\n")
    file.write(f"Test Loss: {loss:.4f}<br>\n")
    file.write(f"Test Accuracy: {accuracy:.4f}<br><br>\n")


5/5 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.7647
Test Loss: 0.6912176012992859
Test Accuracy: 0.7647058963775635


In [125]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted and actual labels from one-hot encoded format to integer labels
y_pred_labels = np.argmax(y_pred, axis=1)
Y_test_labels = np.argmax(Y_test, axis=1)

# Compare predicted labels with actual labels
correct_predictions = np.sum(y_pred_labels == Y_test_labels)
total_predictions = len(Y_test_labels)
accuracy = correct_predictions / total_predictions

# Print accuracy
print("Accuracy:", accuracy)
# Write accuracy to file
with open(f"{metrics_folder}/{filename}.md", "a", encoding="utf-8") as file:
    file.write("Accuracy: {:.4f}\n\n".format(accuracy))


# Write misclassified samples to file
with open(f"{metrics_folder}/{filename}.md", "a", encoding="utf-8") as file:
    file.write("## Misclassified samples:<br>\n")
    for i in range(len(y_pred_labels)):
        if y_pred_labels[i] != Y_test_labels[i]:
            file.write(
                f"Predicted: {label_to_emoji(y_pred_labels[i])} Actual: {label_to_emoji(Y_test_labels[i])}<br>\n"
            )



for i in range(len(y_pred_labels)):
    if y_pred_labels[i]!=Y_test_labels[i]:
        print("Predicted:", label_to_emoji(y_pred_labels[i]), "Actual:", label_to_emoji(Y_test_labels[i]))


5/5 [==============================] - 1s 9ms/step
Accuracy: 0.7647058823529411
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 😂
Predicted: 🔥 Actual: 😃
Predicted: 😍 Actual: 🙏
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 🙏
Predicted: 😠 Actual: 😍
Predicted: 😠 Actual: 😃
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 🙏
Predicted: 😍 Actual: 🙏
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 🙏
Predicted: 😃 Actual: 🔥
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 😂
Predicted: 😃 Actual: 🙏
Predicted: ❤️ Actual: 🙏
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 😂
Predicted: ❤️ Actual: 😂


In [126]:
test = ["I am trying", "I want to cry", "This is just sad"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen=maxlen, padding='post', truncating='post')
y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis=1)

# Write predictions to file
with open(f"{metrics_folder}/{filename}.md", "a", encoding="utf-8") as file:
    file.write("\n## Random test predictions: (accuracy based on discussion)<br>\n")
    for i in range(len(test)):
        file.write(f"{test[i]} {label_to_emoji(y_pred[i])}<br>\n")

1/1 [==============================] - 0s 24ms/step


In [127]:
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I am trying 😭
I want to cry 😭
This is just sad 😭
